In [1]:
import os

#Manejo de matrices y tablas
import numpy as np
import pandas as pd

#Estadistica y funciones matemáticas
import scipy.stats as st
from scipy.optimize import fmin
from scipy import integrate
from scipy.stats.mstats import mquantiles
import statistics 

#Probabilistic programs
import pymc3 as pm
import theano.tensor as tt #NOTA: theano va a cambiar a tensorflow en PyMC4
import theano
from theano.compile.ops import as_op
print('Running on PyMC3 v{}'.format(pm.__version__))

#Graficas
import seaborn as sns
import plotly.graph_objects as go
import altair as alt
from altair_saver import save #ademas instalar en terminal: brew cask install chromedriver
import matplotlib.pyplot as plt
from matplotlib import cm
from matplotlib.gridspec import GridSpec
from matplotlib import animation, rc
from IPython.display import display, HTML, Markdown
import ipywidgets as widgets
from ipywidgets import interact, interactive, fixed, HBox, VBox, Layout
from graphviz import Source, Digraph
import dot2tex as d2t
from latex import build_pdf
from mpl_toolkits.mplot3d import axes3d
import arviz as az

#Funciones propias (tienen que estar en el mismo directorio)
import my_fun as mf

Running on PyMC3 v3.9.3


# Adaptive Toolbox

Santiago Alonso-Díaz, PhD \
Universidad Javeriana

<center><img src="img/7_CB/Gigerenzer1.png" width = "501" height = '500'></center>

Los humanos y animales siempre han dependido de heurísticas para solucionar problemas. 

Ejemplo 1: medir áreas de hendiduras en el piso. Dar una vuelta irregular a la hendidura y dejar feromonas. Dar otra vuelta irregular y estimar el área por la frecuencia que se cruzan los dos caminos.  

<center><img src="img/7_CB/faris-mohammed-unsplash.jpg" width = "301" height = '300'></center>





Ejemplo 2: agarrar objetos a alta velocidad. Mantener un ángulo óptico constante entre uno mismo y el objeto.  

<center><img src="img/7_CB/c-perret-unsplash.jpg" width = "301" height = '300'></center>


¿Por qué aparecen las heurísticas? 
* No omnisciencia (saber todas las variables)
* No omnipotencia (poder computacional infinito)
* Intractabilidad (no hay solución analítica o computable)

Las heurísticas usualmente se relacionan (erroneamente) con el accuracy-effort tradeoff:
* La segunda (o 3era, 4ta, ...) mejor alternativa 
    * no del todo, pueden ser de hecho la mejor. 
* Las usamos por nuestros limítes computacionales 
    * no siempre, incluso en problemas fáciles pueden ser útiles
* Son una alternativa menor por que no usan toda la información, tiempo, y computos 
    * no siempre, e.g. bias-variance trade-off


Las heurísticas pueden ser eficientes

Less-is-more effect:
> "More information or computation can decrease accuracy; therefore, minds rely on simple heuristics in order to be more accurate than strategies that use more information and time." Gigerenzer & Brighton, 2009, pp 110

Veamos algunos casos de less-is-more

<center><div> Tallying</div></center>

### Problema: 

Predecir outcome Y (e.g. oprimir o no oprimir) con atributos X ($X_1, X_2, ... , X_n$)

### Solución: 

Regresión logística (p>0.5 categoria 1, p<0.5 categoria 1):

$$ Y = \beta_0 + \beta X $$


$$ Y = \beta_0 + \beta X $$

¿Cómo obtener $\beta_0$ y $\beta$?

* Maximum likelihood (MLE)
* Aleatorios
* Rankeados por validez (conocimiento del área)
* Todos igual (tallying)

¿Cuál es una solución heurística? Todas menos MLE.

Centremonos en tallying. 

¿Por qué se llama tallying/conteo? Un ejemplo: Y es comprar o no comprar torta de chocolate

$Y_i = 1 + 1*animo + 1*precio_{torta} + 1*colesterol_{torta} + 1*azucar_{torta} + 1*hora_{día}$

Cuando todo tiene el mismo peso (1), la probabilidad crece a medida que aumenta la suma. Es decir, es como contar (tally) las características disponibles.

Hay una versión simple de tallying: take-the-best de diferentes atributos (cues)

A la pregunta cuál tiene más colesterol la persona responde con el mejor cue, es decir, el que considera más relevante y conoce (e.g. calorias en la información nutricional) <br><br>


<center><img src="img/7_CB/Gigerenzer2.svg" width = "501" height = '500'></center>

Czerlinski, Gigerenzer, & Goldstein (1999) pusieron a prueba tallying, take-the-best, y regresión múltiple en veinte dominios. Trataron de predecir los siguientes outcomes:

* Deserción escolar
* Habitante de la calle
* Mortalidad
* Tamaño de ciudad
* Atractivo (hombres)
* Atractivo (mujeres)
* Precio de vivienda
* Renta de la tierra
* Salarios de profesores
* Accidentes de carros
* Consumo de gasolina
* Obesidad a los 18
* Grasa corporal
* Fertilidad de peces
* Tiempo de sueño mamifero
* Calidad de abono de vaca
* Biodiversidad
* Cantidad de lluvia
* Cantidad de oxidantes en Los Angeles
* Cantidad de ozono en San Francisco




Regresión ajusta mejor los datos. Tallying y take-the-best son mejores prediciendo. <br><br><br>

<center><img src="img/7_CB/Gigerenzer3.svg" width = "501" height = '500'></center>

En otro estudio se probaron algunos algoritmos de machine learning en una tarea de adivinar la población de una ciudad. De nuevo, take-the-best le iba bien con muestras pequeñas. <br><br> <br>

<center><img src="img/7_CB/Gigerenzer4.svg" width = "700" height = '700'></center>

Tallying o take-the-best no optimizan nada, son heurísticas. Take-the-best forma parte de un grupo de heurísticas donde less-is-more (less information, more accuracy). Cuestionan la noción que siempre hay un accuracy-effort tradeoff (effort = usar toda la info).

> "Why should a mind waste time and effort in estimating the optimal weights of cues if they do not matter or
even detract from performance?" Gigerenzer & Brighton, 2009, pp. 112

Tres elementos de take-the-best
1. Regla de busqueda: Buscar cues (e.g. grasas saturadas, calorias, proteina, sodio) en orden de validez relativo al outcome (e.g. colesterol).
2. Regla para parar: Cuando se encuentre el primer cue que discrimine entre objetos (e.g. calorias cake vs calorias pie)
3. Regla para decidir: El que tenga el mejor valor en el cue donde se paró.

<center>Ecological rationality</center>

<br>

<center>¿Cuando sirven las heurísticas? Depende pero un principio es el tradeoff entre sesgo-varianza en un ambiente</center>


¿Qué es el tradeoff sesgo-varianza? Es un resultado de inferencia estadística <br> <br>

<center><img src="img/7_CB/bias-and-variance2.jpg" width = "400" height = '400'></center>

$$ Prediction \ error = bias^2 + variance + noise$$

En esta formula (y gráfica), a un nivel de error de predicción deseado, si bajo el sesgo (distancia del modelo a datos recolectados) necesito subir la varianza (distancia del modelo a un nuevo dato) para mantener el error de predicción deseado.



Ejemplo: predecir el clima por el día del año. Un modelo complejo (grado 12) ajusta más puntos (bajo sesgo; gráfica izq.). Sin embargo, unir puntos de un año no asegura que se pueda predecir que pasa un día de otro año (alta varianza; gráfica der.) <br><br>

<center><img src="img/7_CB/Gigerenzer5.svg" width = "900" height = '900'></center>

¿Y qué tiene que ver eso con heurísticas?

En general, tener alta varianza y bajo sesgo puede ser poco adaptativo. Para Gigerenzer & Brighton, un organismo debería apuntar a bajar varianza i.e. mejorar predicción con nuevos datos. Las heurísticas parecen ser buenas para eso. <br> <br>

<center><img src="img/7_CB/Gigerenzer3.svg" width = "401" height = '400'></center>

https://www.sciencedirect.com/science/article/pii/S0010028517303286 